In [1]:
import warnings
warnings.filterwarnings("ignore")

from pyswip import Prolog

import torch
from models.rgcn import RGCN

from prolog.prolog_generation import PrologData

from data.ag.action_genome import AG

In [2]:
# Load dataset
root = '/data/Datasets/ag/'
ag = AG(root)

In [ ]:
# Load model
model = RGCN(ag.num_nodes, ag.num_rels, ag.num_classes, ag.num_bases, ag.num_hidden, ag.num_layers, ag.dropout, ag.use_self_loop)

In [ ]:
# Load prolog data

pd = PrologData('prolog', 'ag', ag, ag.object_classes, ag.relationship_classes, ag.verb_classes, model)